In [1]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D, Dropout, Embedding, GlobalAveragePooling1D, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

import os
import re
import shutil
import string
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print('tensorflow version is {}'.format(tf.__version__))

tensorflow version is 2.8.0


In [3]:
file='stack_overflow_16k'
url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

In [4]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir=file)

In [5]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir='')

In [15]:
os.listdir(dataset)

['test', 'stack_overflow_16k.tar.gz', 'README.md', 'train']

In [16]:
train_dir = os.path.join(dataset, 'train')
train_dir_list = os.listdir(train_dir)

In [17]:
train_dir_list

['python', 'java', 'csharp', 'javascript']

In [18]:
train_dir_list = [i for i in train_dir_list if '.' != i[0]]

In [19]:
train_dir_list

['python', 'java', 'csharp', 'javascript']

In [20]:
indexs = np.random.randint(0, 2000, len(train_dir_list))

for idx, kind in enumerate(train_dir_list):
    file = os.listdir(os.path.join(train_dir, kind))[indexs[idx]]
    file = os.path.join(os.path.join(train_dir, kind), file)
    with open(file) as f:
        print(f.read())
    print('=========================================================')
    

"why isn't my blank code seeing the condition? my blank code isn't working. i have tried many ways of doing it but i can't seem to make this work. could anyone tell me what's wrong?..if not dead:.   time.sleep(2).   print('').   print('""ok next question."" she looked at a paper in front of her.').   time.sleep(2).   print('""where on earth do they a day with 24 hours and a night with 24 hours?""').   time.sleep(2).if ally:.   print('').   print('""small hint, it starts with a and is a cold desert.""').   print('').   answer = input('which place?:'). else:.   answer = input('which place?:').   if answer == 'antarctica' or answer == 'antarctica':.      time.sleep(2).      print('').      print('she nods. ""correct. two more.').   if answer != 'antarctica' and answer != 'antarctica' and ally:.      time.sleep(2).      print('').      print('shaking her head with wide eyes, she looks around carefully.').      time.sleep(2).      print('""the answer is antarctica."" she says, mimicking you

In [23]:
raw_train = tf.keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [35]:
raw_train.class_names

['csharp', 'java', 'javascript', 'python']

In [38]:
ra

AttributeError: 'BatchDataset' object has no attribute 'shape'